In [1]:
import pandas as pd
import numpy as np

In [2]:
from os import listdir
from os.path import isfile, join

path = "../data connectors/wikipedia/"
files = [f for f in listdir(path) if isfile(join(path, f))]

In [3]:
files

['.DS_Store',
 'articles_metrics.json',
 'data.csv',
 'README.md',
 'WikiAPIConnector.py',
 'test api.ipynb']

In [4]:
file_path = path+'data.csv'
d = pd.read_csv(file_path)

In [5]:
d['timestamp'] = d['timestamp'].astype('str')
d['timestamp'] = [pd.Timestamp(year=int(t[:4]),month=int(t[4:6]),day=int(t[6:8])) for t in d['timestamp'].to_list()]

In [6]:
views_col = 'pageview'

feature_names = [views_col,'sma10','sma21']

for article in d['article'].unique():
    data = pd.DataFrame()
    
    d_ = d[d['article'] == article].copy()
    d_['day'] = [pd.Timestamp(t) for t in d_['timestamp']]
    d_.rename(columns={'views':'pageview'},inplace=True)

    x = d_[['article','day',views_col]].copy()

    #generate sma of view
    for f in feature_names[1:]:
        x[f] = x.rolling(window=int(f[-2:]))['pageview'].mean()

    for feature in feature_names:
        x['log_'+feature] = np.log(x[feature])
        x['diff_'+feature] = x[feature].diff()
        x['log_diff_'+feature] = x['log_'+feature].diff()
        x['diff_ratio_'+feature] = (x['diff_'+feature]/x[feature])*100

    data = pd.concat([data,x],ignore_index=True)
    data.to_csv("generated_wiki_pageview_data/"+article+"_daily.csv",index=False)

In [7]:
data

,article,day,pageview,sma10,sma21,log_pageview,diff_pageview,log_diff_pageview,diff_ratio_pageview,log_sma10,diff_sma10,log_diff_sma10,diff_ratio_sma10,log_sma21,diff_sma21,log_diff_sma21,diff_ratio_sma21
0,TRON_(cryptocurrency),2018-01-12,88,NaN,NaN,4.477337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRON_(cryptocurrency),2018-01-13,83,NaN,NaN,4.418841,-5.0,-0.058496,-6.024096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRON_(cryptocurrency),2018-01-14,77,NaN,NaN,4.343805,-6.0,-0.075035,-7.792208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRON_(cryptocurrency),2018-01-15,514,NaN,NaN,6.242223,437.0,1.898418,85.019455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRON_(cryptocurrency),2018-01-16,733,NaN,NaN,6.597146,219.0,0.354922,29.877217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,TRON_(cryptocurrency),2021-03-14,1544,1475.7,1478.523810,7.342132,-50.0,-0.031870,-3.238342,7.296888,12.3,0.008370,0.833503,7.298799,-36.380952,-0.024308,-2.460627
512,TRON_(cryptocurrency),2021-03-15,1374,1493.1,1449.476190,7.225481,-170.0,-0.116650,-12.372635,7.308610,17.4,0.011722,1.165361,7.278958,-29.047619,-0.019842,-2.004008
513,TRON_(cryptocurrency),2021-03-16,1291,1488.9,1436.285714,7.163172,-83.0,-0.062309,-6.429125,7.305793,-4.2,-0.002817,-0.282087,7.269816,-13.190476,-0.009142,-0.918374
514,TRON_(cryptocurrency),2021-03-17,1375,1465.8,1427.571429,7.226209,84.0,0.063037,6.109091,7.290156,-23.1,-0.015636,-1.575931,7.263730,-8.714286,-0.006086,-0.610427
